In [1]:
%load_ext autoreload
%matplotlib widget
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
    run_parallel_post
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
from time import time_ns
from amftrack.util.dbx import upload_folders, load_dbx, download, get_dropbox_folders
from datetime import datetime
from amftrack.pipeline.functions.image_processing.experiment_util import (
    plot_full_image_with_features,
)
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    load_graphs,
)
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    plot_edge_width,
    reconstruct_image_from_general,
    plot_full,
)
from amftrack.pipeline.functions.image_processing.extract_width_fun import (
    get_width_info,
    get_width_info_new,
)

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2022-12-13 12:35:44.306842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 12:35:44.925660: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/home2/cbisot/miniconda3/envs/amftrack/lib/python3.7/site-packages/cv2/../../lib64:
2022-12-13 12:35:44.925709: I tensorflow/stream_executor/cuda/cudart_stub.cc:29

In [2]:
plates = [
    "1014_20220525",
    "1023_20220502",
    "102_20201226",
    "1042_20220523",
    "1045_20220504",
    "1048_20220506",
    "1053_20220413",
    "1076_20220511",
    "127_20220705",
    "130_20220707",
    "152_20201224",
    "30_20220509",
]

In [3]:
directory_targ = directory_project

In [4]:
update_analysis_info(directory_targ)
analysis_info = get_analysis_info(directory_targ)
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [5]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)

analysed:   0%|          | 0/3380 [00:00<?, ?it/s]

In [6]:
# plates = ["1048_20220506"]
folders = all_folders.loc[all_folders["unique_id"].isin(plates)]

In [7]:
analysis_folders = analysis_info.loc[analysis_info["unique_id"].isin(plates)]

In [34]:
analysis_folders

,folder_analysis,version,total_path,Plate,strain,medium,CrossDate,date,unique_id,datetime,...,path_time_hypha_info,path_time_plate_info,path_global_plate_info,date_run_analysis,PrincePos,root,split,Pbait,Temperature,Temp
19,Analysis_1023_20220502_200_Version1,Version1,/projects/0/einf914/data/20220502_1604_Plate15,1023,'C2','001P100N200C',20220502,"02.05.2022, 16:04:",1023_20220502,2022-05-02 16:04:00,...,Analysis_1023_20220502_200_Version1/time_hypha...,Analysis_1023_20220502_200_Version1/time_plate...,Analysis_1023_20220502_200_Version1/global_pla...,"01.01.1970, 01:00:",15,'Carrot','Y','-',25,NaN
24,Analysis_1042_20220523_200_Version1,Version1,/projects/0/einf914/data/20220524_1314_Plate06,1042,'C2','001P100N100C',20220523,"24.05.2022, 13:14:",1042_20220523,2022-05-24 13:14:00,...,Analysis_1042_20220523_200_Version1/time_hypha...,Analysis_1042_20220523_200_Version1/time_plate...,Analysis_1042_20220523_200_Version1/global_pla...,"01.01.1970, 01:00:",6,'Carrot','Y','-',25,NaN
33,Analysis_127_20220705_200_Version1,Version1,/projects/0/einf914/data/20220705_1810_Plate17,127,'C2','001P100N100C',20220705,"05.07.2022, 18:10:",127_20220705,2022-07-05 18:10:00,...,Analysis_127_20220705_200_Version1/time_hypha_...,Analysis_127_20220705_200_Version1/time_plate_...,Analysis_127_20220705_200_Version1/global_plat...,"01.01.1970, 01:00:",17,'Carrot','Y','-',25,NaN
41,Analysis_1076_20220511_200_Version1,Version1,/projects/0/einf914/data/20220511_1541_Plate14,1076,'A5','001P100N200C',20220511,"11.05.2022, 15:41:",1076_20220511,2022-05-11 15:41:00,...,Analysis_1076_20220511_200_Version1/time_hypha...,Analysis_1076_20220511_200_Version1/time_plate...,Analysis_1076_20220511_200_Version1/global_pla...,"01.01.1970, 01:00:",14,'Carrot','Y','-',25,NaN
45,Analysis_1048_20220506_200_Version1,Version1,/projects/0/einf914/data/20220506_2350_Plate03,1048,'C2','001P100N100C',20220506,"06.05.2022, 23:50:",1048_20220506,2022-05-06 23:50:00,...,Analysis_1048_20220506_200_Version1/time_hypha...,Analysis_1048_20220506_200_Version1/time_plate...,Analysis_1048_20220506_200_Version1/global_pla...,"01.01.1970, 01:00:",3,'Carrot','Y','-',25,NaN
57,Analysis_102_20201226_200_Version1,Version1,/scratch-shared/amftrack/stitch_temp2/20201226...,102,'A5','001P100N','20201226',"26.12.2020, 12:24:",102_20201226,2020-12-26 12:24:00,...,Analysis_102_20201226_200_Version1/time_hypha_...,Analysis_102_20201226_200_Version1/time_plate_...,Analysis_102_20201226_200_Version1/global_plat...,"01.01.1970, 01:00:",40,'Carrot','Y',nan,nan,'25'
62,Analysis_130_20220707_200_Version1,Version1,/projects/0/einf914/data/20220707_1452_Plate05,130,'C2','001P100N100C',20220707,"07.07.2022, 14:52:",130_20220707,2022-07-07 14:52:00,...,Analysis_130_20220707_200_Version1/time_hypha_...,Analysis_130_20220707_200_Version1/time_plate_...,Analysis_130_20220707_200_Version1/global_plat...,"01.01.1970, 01:00:",5,'Carrot','Y','-',25,NaN
65,Analysis_30_20220509_200_Version1,Version1,/projects/0/einf914/data/20220510_1437_Plate12,30,'Agg','001P100N200C',20220509,"10.05.2022, 14:37:",30_20220509,2022-05-10 14:37:00,...,Analysis_30_20220509_200_Version1/time_hypha_info,Analysis_30_20220509_200_Version1/time_plate_i...,Analysis_30_20220509_200_Version1/global_plate...,"01.01.1970, 01:00:",12,'Carrot','Y','-',25,NaN
69,Analysis_1014_20220525_100_Version1,Version1,/projects/0/einf914/data/20220525_1932_Plate01,1014,'C2','001P100N100C',20220525,"25.05.2022, 19:32:",1014_20220525,2022-05-25 19:32:00,...,Analysis_1014_20220525_100_Version1/time_hypha...,Analysis_1014_20220525_100_Version1/time_plate...,Analysis_1014_20220525_100_Version1/global_pla...,"01.01.1970, 01:00:",1,'Carrot','Y','-',25,NaN
79,Analysis_1053_20220413_85_Version1,Version1,/projects/0/einf914/data/20220415_1343_Plate07,1053,'C2','001P100N200C',20220413,"15.04.2022, 13:43:",1053_20220413,2022-04-15 13:43:00,...,Analysis_1053_20220413_85_Version1/time_hypha_...,Analysis_1053_20220413_85_Version1/time_plate_...,Analysis_1053_20220413_85_Version

In [ ]:
time = "2:00:00"
name_job = "malin"

list_args = [[]] * len(list_f)
overwrite = True
num_parallel = 6
run_parallel_post(
    "time_edge_post_process.py",
    list_f,
    list_args,
    [directory, overwrite],
    analysis_folders,
    num_parallel,
    time,
    "edge_post_process",
    cpus=32,
    name_job=name_job,
    node="fat",
)

In [ ]:
import pandas as pd
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 20
time = "5:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "recompute_width.py",
            [],
            [],
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "recompute_width",
            cpus=num_cpus,
            name_job="recompute",
            node="fat",
        )

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1935659


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1935660


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


...

Submitted batch job 1935759


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1935760


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: You will be charged for 0.25 node. A full node consists of 128 CPU cores, 983040 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.


Submitted batch job 1935761


In [ ]:
name_job="recompute"

list_f = [
    get_tot_length_C,
    get_tot_length_straight,
    get_time_since_start,
    get_time_since_begin_exp,
    get_width_edge,
    get_pos_x,
    get_pos_y,
]
list_args = [{}] * len(list_f)
# list_args= [[500],[1000],[2000]]+[[]]
# list_args= [[500]]
overwrite = True
load_graphs = True
num_parallel = 32
time = "3:00:00"
for index, row in analysis_folders.iterrows():
    folder = row["folder_analysis"]
    path_time_plate_info = row["path_time_plate_info"]
    plate = row["Plate"]
    num_cpus = 32
    if os.path.isfile(f"{directory_targ}{path_time_plate_info}"):
        whole_plate_info = pd.read_json(
            f"{directory_targ}{path_time_plate_info}", convert_dates=True
        ).transpose()
        whole_plate_info.index.name = "t"
        whole_plate_info.reset_index(inplace=True)
        run_parallel_post(
            "time_edge_post_process.py",
            list_f,
            list_args,
            [directory_targ, overwrite, load_graphs],
            whole_plate_info,
            num_parallel,
            time,
            "edge_post_process",
            cpus=num_cpus,
            name_job=name_job,
            node="fat",
        )
        # break

In [10]:
folders = folders.sort_values(by="datetime")
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,Temp,unique_id,datetime
783,/projects/0/einf914/data/20220506_2350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"06.05.2022, 23:50:",20220506_2350_Plate03,NaN,1048_20220506,2022-05-06 23:50:00
552,/projects/0/einf914/data/20220507_0150_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 01:50:",20220507_0150_Plate03,NaN,1048_20220506,2022-05-07 01:50:00
3050,/projects/0/einf914/data/20220507_0350_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 03:50:",20220507_0350_Plate03,NaN,1048_20220506,2022-05-07 03:50:00
3156,/projects/0/einf914/data/20220507_0550_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 05:50:",20220507_0550_Plate03,NaN,1048_20220506,2022-05-07 05:50:00
2150,/projects/0/einf914/data/20220507_0750_Plate03,1048,3,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"07.05.2022, 07:50:",20220507_0750_Plate03,NaN,1048_20220506,2022-05-07 07:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,/projects/0/einf914/data/20220619_1118_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 11:18:",20220619_1118_Plate30,NaN,1048_20220506,2022-06-19 11:18:00
827,/projects/0/einf914/data/20220619_1718_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 17:18:",20220619_1718_Plate30,NaN,1048_20220506,2022-06-19 17:18:00
939,/projects/0/einf914/data/20220619_2318_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"19.06.2022, 23:18:",20220619_2318_Plate30,NaN,1048_20220506,2022-06-19 23:18:00
1275,/projects/0/einf914/data/20220620_0518_Plate30,1048,30,'Carrot','C2','001P100N100C','Y','-',20220506,25,...,False,True,True,True,True,"20.06.2022, 05:18:",20220620_0518_Plate30,NaN,1048_20220506,2022-06-20 05:18:00


In [11]:
exp = Experiment(directory_targ)
exp.load(folders[:200:30])
exp.dates.sort()

/gpfs/home2/cbisot/pycode/AMFtrack/amftrack/pipeline/functions/image_processing/experiment_class_surf.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.folders["date"], format="%d.%m.%Y, %H:%M:"


2022-05-06 23:50:00
2022-05-09 15:39:00
2022-05-12 05:07:00
2022-05-14 17:52:00
2022-05-17 06:14:00
2022-05-20 10:38:00
2022-05-24 23:55:00


In [12]:
for t in range(exp.ts):
    exp.load_tile_information(t)

In [13]:
for t in range(exp.ts):
    exp.load_tile_information(t)

load_graphs(exp, directory_targ, post_process=True)

In [ ]:
t = 0
plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20)

In [15]:
import networkx as nx
resolution = 50
skip = False
for t in range(exp.ts)[:1]:
    (G, pos) = exp.nx_graph[t], exp.positions[t]
    edge_test = get_width_info_new(exp,t, resolution=resolution, skip=skip)
    nx.set_edge_attributes(G, edge_test, "width")
    exp.nx_graph[t] = G

True


In [16]:
save_graphs(exp)

2022-05-06 23:50:00 True
2022-05-09 15:39:00 True
2022-05-12 05:07:00 True
2022-05-14 17:52:00 True
2022-05-17 06:14:00 True
2022-05-20 10:38:00 True
2022-05-24 23:55:00 True


In [ ]:
t = 2
intervals=[[1, 3], [3, 4], [4, 5], [5, 6],[6,7],[7,9]]

plot_edge_width(exp, t, lambda edge: edge.width(t), dilation=20,intervals= intervals)